In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = 'dataset/train',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=16,
    image_size=(256, 256),
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = 'dataset/val',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=16,
    image_size=(256, 256),
)

Found 11669 files belonging to 3 classes.
Found 2500 files belonging to 3 classes.


In [3]:
# To test model on small scale

# subset_size = 200
# train_ds = train_ds.take(subset_size)
# validation_ds = validation_ds.take(subset_size)

In [4]:
# Normalize
def process(image, label):
    image = tf.cast(image/255. , tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
from keras import layers, models

input_shape = (256, 256, 3)
inputs = tf.keras.Input(shape=input_shape)

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

attention = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)
x = layers.Multiply()([x, attention])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(3, activation='softmax')(x)

model = models.Model(inputs=inputs, outputs=outputs)


In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 256, 256, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 256, 256, 32  128        ['conv2d_4[0][0]']               
 rmalization)                   )                                                           

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,              # wait for 2 epochs without improvement
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_model_4drop.h5',
    save_best_only=True,
    monitor='val_loss'
)

| Dropout          | Test Accuracy | Validation Accuracy |
|------------------|---------------|---------------------|
| With Dropout (0.3) | 85%          | 87%                 |
| Without Dropout   | 84%           | 86%                 |
| With Dropout (0.2) | 90%          | 91%                 |


In [15]:
history = model.fit(train_ds , epochs=10, validation_data = validation_ds, callbacks=[early_stopping, checkpoint])

Epoch 1/10


730/730 [==============================] - 96s 130ms/step - loss: 0.6219 - accuracy: 0.7199 - val_loss: 2.8274 - val_accuracy: 0.4452
Epoch 2/10
730/730 [==============================] - 135s 185ms/step - loss: 0.4130 - accuracy: 0.8256 - val_loss: 0.9406 - val_accuracy: 0.6276
Epoch 3/10
730/730 [==============================] - 140s 192ms/step - loss: 0.3115 - accuracy: 0.8741 - val_loss: 0.2286 - val_accuracy: 0.9168
Epoch 4/10
730/730 [==============================] - 152s 209ms/step - loss: 0.2440 - accuracy: 0.9047 - val_loss: 0.5329 - val_accuracy: 0.7916
Epoch 5/10
730/730 [==============================] - 151s 207ms/step - loss: 0.2162 - accuracy: 0.9212 - val_loss: 0.4274 - val_accuracy: 0.8400


In [10]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = 'dataset/test',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=16,
    image_size=(256, 256),
)

test_ds = test_ds.map(process)


Found 2504 files belonging to 3 classes.


In [16]:
loss, accuracy = model.evaluate(test_ds)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

157/157 [==============================] - 6s 34ms/step - loss: 0.2632 - accuracy: 0.9046
Test Loss: 0.26318761706352234
Test Accuracy: 0.904552698135376


In [5]:
model = keras.models.load_model('best_model_2drop.h5')

In [12]:
import cv2
import numpy as np
from PIL import Image
def predict_image(image):
    # Convert the PIL image to a format suitable for prediction
    img_rgb = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (256, 256))
    img_input = img_resized.reshape((1, 256, 256, 3))
    
    # Normalize the image
    img_input = img_input / 255.0

    # Predict
    prediction = model.predict(img_input)
    
    # Determine label and confidence
    return prediction
print(predict_image(Image.open(r'dataset\test\general\2.jpg')))

1/1 [==============================] - 0s 36ms/step
[[0.6703616  0.04246514 0.28717327]]
